# RBC Loading & Modeling Template

This notebook demonstrates how to:

1. Load participant metadata from the Reproducible Brain Charts (RBC).  
2. Retrieve FreeSurfer region-surface stats for participants.  
3. Clean and filter data.  
4. Plot simple histograms of the target variable.  
5. Prepare a subset of subjects and features for modeling.


### Import necessary packages

In [1]:
# ---------------------------------------------------------------------------
# Imports
# ---------------------------------------------------------------------------

from pathlib import Path             # for handling filesystem paths
import re                            # for regex string cleaning
import numpy as np                   # for numeric operations
import pandas as pd                  # for tabular data handling
from rbclib import RBCPath           # RBC-specific path handling



### Data paths & participant metadata

In [2]:
# ---------------------------------------------------------------------------
# Paths to the RBC data
# ---------------------------------------------------------------------------
rbcdata_path = Path("/home/jovyan/shared/data/RBC")
train_filepath = rbcdata_path / "train_participants.tsv"
test_filepath  = rbcdata_path / "test_participants.tsv"

# ---------------------------------------------------------------------------
# Load participant metadata from TSV files
# ---------------------------------------------------------------------------

# Load train and test participants
train_data = pd.read_csv(train_filepath, sep="\t")
test_data  = pd.read_csv(test_filepath,  sep="\t")

# Concatenate into one DataFrame
all_data = pd.concat([train_data, test_data], ignore_index=True)

all_data.head()


,participant_id,study,study_site,session_id,wave,age,sex,race,ethnicity,bmi,handedness,participant_education,parent_1_education,parent_2_education,p_factor,internalizing_mcelroy_harmonized_all_samples,externalizing_mcelroy_harmonized_all_samples,attention_mcelroy_harmonized_all_samples,cubids_acquisition_group
0,1000393599,PNC,PNC1,PNC1,1,15.583333,Male,Black,not Hispanic or Latino,22.15,Right,9th Grade,Complete primary,Complete secondary,0.589907,-0.449373,-0.630780,-1.842178,1
1,1001970838,PNC,PNC1,PNC1,1,17.833333,Male,Other,Hispanic or Latino,23.98,Right,11th Grade,Complete tertiary,Complete tertiary,-0.659061,0.531072,0.392751,0.190706,1
2,1007995238,PNC,PNC1,PNC1,1,13.750000,Female,Other,not Hispanic or Latino,23.77,Right,6th Grade,Complete tertiary,Complete primary,-1.608375,-0.744118,-0.314187,-0.432662,1
3,1011497669,PNC,PNC1,PNC1,1,16.666667,Male,White,not Hispanic or Latino,29.68,Right,9th Grade,Complete tertiary,Complete tertiary,-1.233807,-0.896835,-0.449099,0.111167,1
4,1017092387,PNC,PNC1,PNC1,1,18.666667,Female,Black,not Hispanic or Latino,23.24,Right,11th Grade,Complete primary,Complete primary,-0.923100,-0.313455,2.204168,-0.782266,1


### FreeSurfer helper functions

In [5]:
# ---------------------------------------------------------------------------
# FreeSurfer utility functions
# ---------------------------------------------------------------------------

def load_fsdata_raw(participant_id, local_cache_dir=Path.home() / "cache"):
    """
    Load the raw FreeSurfer TSV file for a PNC participant.
    
    Parameters
    ----------
    participant_id : str or int
        RBC participant identifier (without 'sub-' prefix).
    local_cache_dir : Path
        Local cache directory where RBC files will be stored.
    
    Returns
    -------
    pd.DataFrame
        Long-form FreeSurfer stats for the participant.
    """
    local_cache_dir = Path(local_cache_dir)
    local_cache_dir.mkdir(exist_ok=True)

    pnc_fspath = RBCPath(
        "rbc://PNC_FreeSurfer/freesurfer",
        local_cache_dir=local_cache_dir
    )
    subdir = pnc_fspath / f"sub-{participant_id}"
    tsv_path = subdir / f"sub-{participant_id}_regionsurfacestats.tsv"

    return pd.read_csv(tsv_path, sep="\t")


def filter_by_atlas(df, atlas_substr):
    """Filter rows by atlas substring (case-insensitive)."""
    mask = df["atlas"].astype(str).str.contains(atlas_substr, case=False, na=False)
    if not mask.any():
        available = sorted(df["atlas"].dropna().unique())
        raise ValueError(
            f"No atlas rows contained '{atlas_substr}'. "
            f"Available atlases: {', '.join(available)}"
        )
    return df.loc[mask].copy()


def select_measure(df, measure):
    """Select only the relevant columns for a given brain measure."""
    cols = ["subject_id", "atlas", "hemisphere", "StructName", measure]
    missing = [c for c in cols if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns: {missing}")
    return df[cols].copy()


def sanitize(col):
    """Sanitize a string for safe use as a column name."""
    clean = re.sub(r"[^0-9A-Za-z]+", "_", col)   # replace non-alphanumeric chars
    clean = re.sub(r"_{2,}", "_", clean)         # collapse multiple underscores
    return clean.strip("_")                      # strip leading/trailing underscores


def pivot(df, measure):
    """
    Pivot long-form FreeSurfer stats into a wide format with one row per subject.
    """
    subj = df["subject_id"].iloc[0]
    out = {"subject_id": subj}
    for _, row in df.iterrows():
        raw = f"{row['hemisphere']}_{row['atlas']}_{row['StructName']}_{measure}"
        col = sanitize(raw)
        out[col] = row[measure]
    return pd.DataFrame([out])


def load_and_pivot_fsdata(participant_id, atlas, measure,
                          local_cache_dir=Path.home() / "cache"):
    """
    Load, filter, select, and pivot FreeSurfer stats into a wide format.
    """
    fsdata_raw = load_fsdata_raw(participant_id, local_cache_dir)
    fsdata_by_atlas = filter_by_atlas(fsdata_raw, atlas)
    fsdata_by_measure = select_measure(fsdata_by_atlas, measure)
    return pivot(fsdata_by_measure, measure)


### Pick atlas & measure

In [ ]:
# ---------------------------------------------------------------------------
# Pick an atlas and brain measure for modeling
# ---------------------------------------------------------------------------

ATLAS="aparc.DKTatlas"
MEASURE="ThickAvg"


# We'll display a progress bar `prog` as we go also:
from ipywidgets import IntProgress
prog = IntProgress(min=0, max=len(all_data))
display(prog)

demo = [
    'age',
    'sex',
    'race',
    'ethnicity',
    'bmi',
    'handedness',
    'participant_education',
    'parent_1_education',
    'parent_2_education',
    'p_factor'
]


records = []

for row in all_data.itertuples(index=False):
    # start record with participant_id + all demo vars
    rec = {col: getattr(row, col) for col in ['participant_id'] + demo}

    # try to load & pivot; if successful, merge in all FS cols
    try:
        freesurfer = (
                        load_and_pivot_fsdata(
                            participant_id=rec['participant_id'],
                            atlas=ATLAS,
                            measure=MEASURE,
                            local_cache_dir=Path.home()/"cache"
                        )
            .drop(columns='subject_id')  # remove duplicate ID col
            .iloc[0]
            .to_dict()
        )
        rec.update(freesurfer)
    except (FileNotFoundError, ValueError):
        # leave rec with only ID+demo if FS data is missing/blank
        pass

    records.append(rec)
    prog.value += 1

all_demo_and_brain = pd.DataFrame(records)

# split into train/test
train_vars = all_demo_and_brain[all_demo_and_brain['p_factor'].notna()]
test_vars  = all_demo_and_brain[all_demo_and_brain['p_factor'].isna()]

all_demo_and_brain.head()

IntProgress(value=0, max=1601)

# Data cleaning

### Remove uninformative subjects (with plots)

In [ ]:
# ---------------------------------------------------------------------------
# Remove subjects at the minimum p_factor value and plot distributions
# ---------------------------------------------------------------------------

# Plot histogram of p_factor for all training subjects
train_data["p_factor"].hist(bins=100)

# Print minimum value and its count
print("Minimum p_factor:", train_data["p_factor"].min())
print("Number of subjects with min value:",
      train_data["p_factor"].value_counts()[train_data["p_factor"].min()])

# Remove subjects with minimum p_factor (to avoid bias in modeling)
min_val_idx = train_data["p_factor"] == train_data["p_factor"].min()
removed_train_data = train_data[min_val_idx].copy()
clean_train_data   = train_data[~min_val_idx]

# Plot histogram after cleaning
clean_train_data["p_factor"].hist()
